In [1]:
import pandas as pd
import numpy as np
import subprocess
import collections
import os

## FDR correction After gene level adjustment

path = "/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/"

def fdrcorrection(tissue):
    print(tissue, ' variants ...')
#Get most signif. variant by gene from linear reg STRs
    LR1=pd.read_csv(path+ tissue+"/chr21_imp.tab" , '\t')
    
#Locus level
    LR1['p.wald'].to_csv('pvalues.txt', sep='\n', index=False)
    Tell = subprocess.call("/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/PostRegressionFixes/fdr-correct.r")
    Qval=pd.read_csv('/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/PostRegressionFixes/qvalues.txt', sep=' ')
    
    LR1['llqvalue']=list(Qval['qvalue'])
    LR1['llsignif']=list(Qval['significant'])

#Gene level
    LR0 = LR1.sort_values("p.wald").groupby("gene", as_index=False).first()     
    print(LR1.shape, '  to  ', LR0.shape)

    #Add counts tests by gene
    counts=pd.DataFrame({'cts' : LR1.groupby(["gene"]).size()})    ## This is the count by genes
    genes = list(LR0['gene'])
    LR0['NTEST']= list(counts.loc[genes]['cts'])
    
    #Gene level adjustment
    #(1) min_pval* #test
    LR0['AD.pval']=LR0['p.wald']*LR0['NTEST']
    #(2) if AD_pval>1 => AD_pval=1
    LR0['AD.pval'][LR0['AD.pval']>1] = 1
    
    #Save pval in file and FDR correct
    LR0['AD.pval'].to_csv('pvalues.txt', sep='\n', index=False)
    Tell = subprocess.call("/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/PostRegressionFixes/fdr-correct.r")
        
    #FDR corrected... add to dataframe
    Qval=pd.read_csv('/home/szfeupe/projects/GTEX_eSTRs/gtex-estrs/Scripts/PostRegressionFixes/qvalues.txt', sep=' ')
    LR0['qvalue']=list(Qval['qvalue'])
    LR0['significant']=list(Qval['significant'])

#Merging
    merging=['gene','chrom','str.id','str.start','beta','beta.se','p.wald','llqvalue','llsignif']
    LRP = pd.merge(LR1,LR0, on=merging, how='left')
    
#Header arrangement
    Head=['gene','chrom','str.id','str.start','p.wald','llqvalue','llsignif','NTEST','qvalue','significant','beta','beta.se']
    Out=LRP[Head]
    Out.to_csv(path+tissue+'/47PQValues', sep='\t', index=False)

    S=LR0['AD.pval']
    print(len(S),' total tests... ', len(S[S>=1]) , ' pvalues were reduced to 1')
    print(len(LRP[LRP['qvalue'] <=0.1]),'\t gene level qval<=0.1')
    print(len(LRP[LRP['llqvalue'] <=0.1]),'\t locus level qval<=0.1')
    print(len(LRP[LRP['llqvalue'] <=0.01]),'\t qval<0.01\n')
    return()


fdrcorrection("Imputation_STR")

Imputation_STR  variants ...
(8629, 14)   to   (164, 14)


/usr/lib/python3.4/site-packages/ipykernel/__main__.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


164  total tests...  31  pvalues were reduced to 1
73 	 gene level qval<=0.1
947 	 locus level qval<=0.1
466 	 qval<0.01



()

In [ ]:

#NON 47
989  total tests...  296  pvalues were reduced to 1
273 	 gene level qval<=0.1
4371 	 locus level qval<=0.1
2071 	 qval<0.01

Non Imputed There were 158 eSTRs

By eGenes (This are gene influenced by an eSTR)
Overlap between imputed and non imputed eSTRs
(136 egenes) 
Overlap between imputed @47 and non imputed eSTRs
(136 egenes)

By eSTRs (These are pairs Gene+eSTRs)
Overlap between imputed and non imputed eSTRs
(15 eSTRs to specific gene)
Overlap between imputed @47 and non imputed eSTRs
(12 eSTRs to specific gene)

In [14]:
puted = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Imputation_STR/PQValues', sep='\t')
#put47 = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Imputation_STR/47PQValues', sep='\t')
lessnoted = pd.read_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/Imputation_STR/Non_imputed_PQvalues', sep='\t')

puted = puted.loc[puted['qvalue']<=0.1]
put47 = put47.loc[put47['qvalue']<=0.1]
noted = noted.loc[noted['qvalue']<=0.1]

print(puted.shape, put47.shape, noted.shape)

print("Overlap between imputed and non imputed eSTRs")
over1 = puted.loc[puted['str.id'].isin(list(noted['str.id']))]
print (over1.shape)

print("Overlap between imputed @47 and non imputed eSTRs") 
over2 = put47.loc[put47['str.id'].isin(list(noted['str.id']))]
print(over2.shape)

noted.loc[noted['gene'].isin(list(over1['gene']))].sort_values('gene')

(273, 12) (274, 12) (158, 12)
Overlap between imputed and non imputed eSTRs
(15, 12)
Overlap between imputed @47 and non imputed eSTRs
(12, 12)


,gene,chrom,str.id,str.start,p.wald,llqvalue,llsignif,NTEST,qvalue,significant,beta,beta.se
302,ENSG00000069188.12,chr17,STR_71591341,71591341,1.682718e-04,0.005669,1,47.0,0.062860,1.0,0.229551,0.060130
1202,ENSG00000125447.12,chr17,STR_73194441,73194441,1.465873e-06,0.000833,1,25.0,0.001827,1.0,0.292934,0.059411
1269,ENSG00000125450.6,chr17,STR_73213462,73213462,1.289477e-13,0.000833,1,27.0,0.001756,1.0,-0.450806,0.057380
2069,ENSG00000141519.10,chr17,STR_78020332,78020332,5.808892e-25,0.000833,1,16.0,0.001756,1.0,0.588419,0.051036
2136,ENSG00000141527.12,chr17,STR_78178427,78178427,1.655825e-04,0.005606,1,22.0,0.034370,1.0,0.233737,0.061128
2377,ENSG00000141570.6,chr17,STR_77784458,77784458,7.012926e-06,0.000878,1,17.0,0.002556,1.0,0.274099,0.059757
2677,ENSG00000157637.8,chr17,STR_79275829,79275829,1.951884e-06,0.000833,1,12.0,0.001765,1.0,-0.293415,0.060221
2688,ENSG00000159199.9,chr17,STR_47015292,47015292,5.239589e-06,0.000849,1,3.0,0.001756,1.0,0.284672,0.061120
2792,ENSG00000161542.12,chr17,STR_74293588,74293588,1.313021e-08,0.000833,1,22.0,0.001756,1.0,0.343405,0.058471
2968,ENSG00000167100.10,chr17,STR_48208033,48208033,2.828561e-06,0.000833,1,4.0,0.001756,1.0,-0.315963,0.065628


In [15]:
over1.sort_values('gene')

,gene,chrom,str.id,str.start,p.wald,llqvalue,llsignif,NTEST,qvalue,significant,beta,beta.se
39052,ENSG00000069188.12,chr17,STR_71591341,71591341,1.117955e-04,0.002810,1,103.0,0.048463,1.0,-0.249882,0.063570
24358,ENSG00000108786.6,chr17,STR_40795105,40795105,1.339152e-04,0.003232,1,35.0,0.022943,1.0,-0.247106,0.063617
31253,ENSG00000108819.9,chr17,STR_48208033,48208033,5.633338e-04,0.010362,1,34.0,0.073995,1.0,-0.223780,0.063988
40756,ENSG00000125447.12,chr17,STR_73194441,73194441,1.392427e-06,0.000892,1,86.0,0.007422,1.0,0.309374,0.062432
40693,ENSG00000125450.6,chr17,STR_73213462,73213462,2.412673e-11,0.000892,1,80.0,0.007422,1.0,-0.418618,0.059624
45126,ENSG00000141519.10,chr17,STR_78020332,78020332,5.068966e-28,0.000892,1,59.0,0.007422,1.0,0.636869,0.050617
45302,ENSG00000141527.12,chr17,STR_78178427,78178427,1.657298e-05,0.000974,1,58.0,0.008802,1.0,-0.277474,0.063075
44986,ENSG00000141570.6,chr17,STR_77784458,77784458,1.019192e-05,0.000905,1,37.0,0.007687,1.0,-0.284048,0.062949
45963,ENSG00000157637.8,chr17,STR_79275829,79275829,1.095871e-06,0.000892,1,38.0,0.007422,1.0,0.312266,0.062370
30143,ENSG00000159199.9,chr17,STR_47015292,47015292,6.536880e-06,0.000892,1,65.0,0.007687,1.0,0.289911,0.062834
